In [2]:
import xgboost
import numpy as np
import pandas as pd
import xgboost as xgb
#import sklearn
from sklearn.model_selection import train_test_split
#记录程序运行时间
import time 
start_time = time.time()

#读入数据
train = pd.read_csv("data/digit-recognizer/train.csv")
tests = pd.read_csv("data/digit-recognizer/test.csv") 

#用sklearn.cross_validation进行训练数据集划分，这里训练集和交叉验证集比例为7：3，可以自己根据需要设置
train_xy,val = train_test_split(train, test_size = 0.3,random_state=1)
#print(train_xy)

y = train_xy.label
X = train_xy.drop(['label'],axis=1)
val_y = val.label
val_X = val.drop(['label'],axis=1)

#xgb矩阵赋值
xgb_val = xgb.DMatrix(val_X,label=val_y)
xgb_train = xgb.DMatrix(X, label=y)
xgb_test = xgb.DMatrix(tests)


In [6]:
params={
'booster':'gbtree',
'objective': 'multi:softmax', #多分类的问题
'num_class':10, # 类别数，与 multisoftmax 并用
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':1000,
'nthread':7,# cpu 线程数
#'eval_metric': 'auc'
}
plst = list(params.items())
print(plst)
num_rounds = 10 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)
model.save_model('model/xgb.model') # 用于存储训练出的模型
print("best best_ntree_limit %s"%(model.best_ntree_limit))

[('booster', 'gbtree'), ('objective', 'multi:softmax'), ('num_class', 10), ('gamma', 0.1), ('max_depth', 12), ('lambda', 2), ('subsample', 0.7), ('colsample_bytree', 0.7), ('min_child_weight', 3), ('silent', 0), ('eta', 0.007), ('seed', 1000), ('nthread', 7)]
[0]	train-merror:0.088095	val-merror:0.12381
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 100 rounds.
[1]	train-merror:0.062891	val-merror:0.098889
[2]	train-merror:0.05517	val-merror:0.088889
[3]	train-merror:0.051973	val-merror:0.081349
[4]	train-merror:0.046871	val-merror:0.076667
[5]	train-merror:0.046156	val-merror:0.074762
[6]	train-merror:0.045646	val-merror:0.076032
[7]	train-merror:0.043844	val-merror:0.07381
[8]	train-merror:0.043027	val-merror:0.073175
[9]	train-merror:0.042313	val-merror:0.072381
best best_ntree_limit 10


In [8]:
preds = model.predict(xgb_test,ntree_limit=model.best_ntree_limit)

np.savetxt('data/xgb_submission.csv',np.c_[range(1,len(tests)+1),preds],delimiter=',',header='ImageId,Label',comments='',fmt='%d')

#输出运行时长
cost_time = time.time()-start_time
print("xgboost success!",'\n',"cost time:",cost_time,"(s)......")

xgboost success! 
 cost time: 754.6263167858124 (s)......


In [9]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix('data/agaricus.txt.train')
dtest = xgb.DMatrix('data/agaricus.txt.test')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
print(preds)

[15:33:03] 6513x127 matrix with 143286 entries loaded from data/agaricus.txt.train
[15:33:03] 1611x127 matrix with 35442 entries loaded from data/agaricus.txt.test
[0.28583017 0.9239239  0.28583017 ... 0.9239239  0.05169873 0.9239239 ]


xgboost的参数学习

params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':10,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':12,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':0.7, # 随机采样训练样本
    'colsample_bytree':3,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.007,  # 如同学习率
    'seed':1000,
    'nthread':7,  #CPU线程数
    #'eval_metric':'auc'
}

eta [default=0.3] 
为了防止过拟合，更新过程中用到的收缩步长。在每次提升计算之后，算法会直接获得新特征的权重。
eta通过缩减特征的权重使提升计算过程更加保守。缺省值为0.3
取值范围为：[0,1]
通常最后设置eta为0.01~0.2

gamma值使得算法更conservation，且其值依赖于loss function ，在模型中应该进行调参。

max_depth [default=6] 
树的最大深度。缺省值为6
取值范围为：[1,∞]
指树的最大深度
树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合
建议通过交叉验证（xgb.cv ) 进行调参
通常取值：3-10

min_child_weight [default=1] 
孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。



In [ ]:
xgboost.train(params,
              dtrain,
              num_boost_round=10,
              evals(),
              obj=None,
              feval=None,
              maximize=False,
              early_stopping_rounds=None,
              evals_result=None,
              verbose_eval=True,
              learning_rates=None,
              xgb_model=None)

parms：这是一个字典，里面包含着训练中的参数关键字和对应的值，形式是parms = {'booster':'gbtree','eta':0.1}

dtrain：训练的数据

num_boost_round：这是指提升迭代的个数

evals：这是一个列表，用于对训练过程中进行评估列表中的元素。形式是evals = [(dtrain,'train'),(dval,'val')] 或者是 evals =[(dtrain,'train')] ，对于第一种情况，它使得我们可以在训练过程中观察验证集的效果。

obj ：自定义目的函数

feval：自定义评估函数

maximize：是否对评估函数进行最大化

early_stopping_rounds：早起停止次数，假设为100，验证集的误差迭代到一定程度在100次内不能再继续降低，就停止迭代。这要求evals里至少有一个元素，如果有多个，按照最后一个去执行。返回的是最后的迭代次数（不是最好的）。如果early_stopping_rounds存在，则模型会生成三个属性，bst.best_score ,bst.best_iteration和bst.best_ntree_limit

evals_result：字典，存储在watchlist中的元素的评估结果

verbose_eval（可以输入布尔型或者数值型）：也要求evals里至少有一个元素，如果为True，则对evals中元素的评估结果会输出在结果中；如果输入数字，假设为5，则每隔5个迭代输出一次。

learning_rates：每一次提升的学习率的列表

xgb_model：在训练之前用于加载的xgb_model

第一步：确定学习速率和tree_based参数调优的估计器数目
　　为了确定Boosting参数，我们要先给其他参数一个初始值。咱们先按照如下方法取值：

1，max_depth = 5：这个参数的取值最好在3-10之间，我选的起始值为5，但是你可以选择其他的值。起始值在4-6之间都是不错的选择。
2，min_child_weight = 1 ：这里选择了一个比较小的值，因为这是一个极不平衡的分类问题。因此，某些叶子节点下的值会比较小。
3，gamma = 0 :起始值也可以选择其它比较小的值，在0.1到0.2之间就可以，这个参数后继也是要调整的。
4，subsample,colsample_bytree = 0.8  这个是最常见的初始值了。典型值的范围在0.5-0.9之间。
5，scale_pos_weight =1 这个值时因为类别十分不平衡。
　　注意，上面这些参数的值知识一个初始的估计值，后继需要调优。这里把学习速率就设成默认的0.1。然后用Xgboost中的cv函数来确定最佳的决策树数量。
  
第二步：max_depth和min_weight参数调优
　　我们先对这两个参数调优，是因为他们对最终结果有很大的影响。首先，我们先大范围地粗略参数，然后再小范围的微调。

　　注意：在这一节我会进行高负荷的栅格搜索（grid search），这个过程大约需要15-30分钟甚至更久，具体取决于你系统的性能，你也可以根据自己系统的性能选择不同的值。
  
第三步：gamma参数调优
　　在已经调整好其他参数的基础上，我们可以进行gamma参数的调优了。Gamma参数取值范围很大，这里我们设置为5，其实你也可以取更精确的gamma值。

第四步：调整subsample 和 colsample_bytree参数
　　尝试不同的subsample 和 colsample_bytree 参数。我们分两个阶段来进行这个步骤。这两个步骤都取0.6,0.7,0.8,0.9作为起始值。

第五步：正则化参数调优
　　由于gamma函数提供了一种更加有效的降低过拟合的方法，大部分人很少会用到这个参数，但是我们可以尝试用一下这个参数。

第六步：降低学习速率
　　最后，我们使用较低的学习速率，以及使用更多的决策树，我们可以用Xgboost中CV函数来进行这一步工作。


In [ ]:
DMatrix 查看数据情况
dtrain.num_col()
dtrain.num_row()